In [2]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup

from sqlalchemy import create_engine
from config import Config
import pandas as pd

In [41]:
BASE_URL = "https://blokkenschema.nl"

In [14]:
res = r.get(BASE_URL+ '/lowlands-2023/line-up')

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')
hl_links = [li.a['href'] for li in soup.find_all('li', class_='headliner')]

In [92]:
def get_schema_data(link):
    res_hl  = r.get(BASE_URL + link)
    soup_hl = BeautifulSoup(res_hl.text, 'html.parser')
    hl_data = soup_hl.find('div', class_='performance-data')
    return {
        'name': soup_hl.find('div', itemprop='performer').find('h3').string,
        'location': hl_data.find('div', itemprop='location').string,
        'start_date': hl_data.find('div', itemprop='startDate')['content'],
        'end_date': hl_data.find('span', itemprop='endDate')['content'],
    }

In [94]:
list_of_schema = [get_schema_data(link) for link in hl_links]
df = pd.DataFrame(list_of_schema)
df.to_csv("lowlands_schema.csv")

In [29]:
cfg = Config()
engine = create_engine(cfg.SQLALCHEMY_DATABASE_URI, echo=True)
df.to_sql('artist', con=engine, if_exists='append', index=False)